In [1]:
from adlframework.retrievals.BlobLocalCache import BlobLocalCache
from adlframework.datasource import DataSource
from adlframework.dataentity.arff_de import ARFFDataEntity
# from adlframework.dataentity.audio_de import AudioRecordingDataEntity
from adlframework.experiment import SimpleExperiment
# from keras.losses import KLD, MAE
# from keras.optimizers import Adadelta, Adam
# from nets import dense_network, deep_dense_network
# import pdb
# import numpy as np
# from keras.callbacks import ModelCheckpoint

AJAJA


Using TensorFlow backend.


### Custom Controllers/Filters

In [2]:
def remove_wavs(entity):
    '''
    Filter out wav and extraneous files
    '''
    return entity.unique_id[-4:] == 'arff'

def split_to_features(sample):
    f, label = sample
    f.next() # Remove empty row
    data = np.array([x for x in f.next()][1:])
    try:
        f.next() # Close file
        del f
    except StopIteration:
        pass
    return data, label

### Metadata

In [3]:
cache_path = 'local_cache/Segmented_AVEC/'
train_retrieval = BlobLocalCache(cache_path+'wav/train', cache_path+'labels/train')
val_retrieval = BlobLocalCache(cache_path+'wav/val', cache_path+'labels/val')
test_retrieval = BlobLocalCache(cache_path+'wav/test', cache_path+'labels/test')
epochs = 100
max_mem = .3

controllers = [split_to_features]
prefilters = [remove_wavs]

Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.


### Data Sources

In [4]:
## Creating and splitting datasets
train_ds = DataSource(train_retrieval, ARFFDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
						controllers=controllers,
						prefilters=prefilters,
                        max_mem_percent=max_mem)

val_ds = DataSource(val_retrieval, ARFFDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
						controllers=controllers,
						prefilters=prefilters,
                        max_mem_percent=max_mem)

test_ds = DataSource(test_retrieval, ARFFDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
						controllers=controllers,
						prefilters=prefilters,
                        max_mem_percent=max_mem)

INFO:adlframework.datasource:Prefiltering entities
INFO:adlframework.datasource:----------------------------------------Filter 0----------------------------------------
INFO:adlframework.datasource:<function remove_wavs at 0x7fdf5f9c09b0>
INFO:adlframework.datasource:Before filter 0, there are 209972 entities.
100%|██████████| 209972/209972 [00:00<00:00, 2621800.55it/s]
INFO:adlframework.datasource:After filter 0, there are 104986 entities.
INFO:adlframework.datasource:Prefiltering entities
INFO:adlframework.datasource:----------------------------------------Filter 0----------------------------------------
INFO:adlframework.datasource:<function remove_wavs at 0x7fdf5f9c09b0>
INFO:adlframework.datasource:Before filter 0, there are 59992 entities.
100%|██████████| 59992/59992 [00:00<00:00, 2490002.23it/s]
INFO:adlframework.datasource:After filter 0, there are 29996 entities.
INFO:adlframework.datasource:Prefiltering entities
INFO:adlframework.datasource:----------------------------------

In [5]:
### Callbacks
callbacks = [ModelCheckpoint('weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5')]

In [6]:
### Network
net = deep_dense_network(input_shape=(989, ),
					target_shape=(3))

### Experiment

In [7]:
exp = SimpleExperiment(train_datasource=train_ds,
						validation_datasource=val_ds,
                       test_datasource=test_ds,
						loss=MAE,
						optimizer=Adam(),
						metrics=['mae'],
						network = net,
						callbacks=callbacks,
                        epochs=epochs)

In [8]:
exp.run()

INFO:adlframework.experiment:Compiled network.


Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 989)               3956      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               506880    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 578,935
Trainable params: 575,933
Non-trainable params: 3,002
_________________________________________________________________
None
Epoch 1/100
30/30 [==============================] - 7s - loss: 0.1368 - mean_absolute_error: 0.1368 - val_loss:

KeyboardInterrupt: 